In [1]:
!pip install spotipy

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="b9155a56e0f54f0fbb99dae35eb54732",
                                                           client_secret="65e11367dd3848aeb711ba8cb5f9dc50"))

In [3]:
import pandas as pd

To move forward with the project, you need to create a collection of songs with their audio features - as large as possible!

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster. The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

In [4]:
#function in case the playlist has more than 100 songs
def get_playlist_tracks(username, playlist_id):

    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    return tracks

In [5]:
#function to get song titles & song ids:
def get_songs_and_ids_from_playlist(playlist_id):

    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)

    songs = []
    songs_ids = []

    for track in tracks_from_playlist:
        songs.append(track['track']['name'])
        songs_ids.append(track['track']['uri'])

    return list(songs), list(songs_ids)

In [6]:
#function to get artists and artists_ids
def get_artists_and_ids_from_playlist(playlist_id):

    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)

    artists = []
    artists_ids = []

    for track in tracks_from_playlist:
        artists_info = track['track']['artists']

        for artist_info in artists_info:
            artists.append(artist_info['name'])
            artists_ids.append(artist_info['id'])

    return list(set(artists)), list(set(artists_ids))

In [7]:
#function to get top songs from all the artists from the list
def get_top_songs_from_artists_from_playlist(playlist_id):
    
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    artists_ids = []
    
    for track in tracks_from_playlist:
        artists_info = track['track']['artists']

        for artist_info in artists_info:
            artists_ids.append(artist_info['id'])

    song_titles = []
    song_uri = []

    for artist_id in artists_ids:
        results = sp.artist_top_tracks(artist_id, country='US')
        top_tracks = results['tracks']
        
        for top_track in top_tracks:
            song_titles.append(top_track['name'])
            song_uri.append(top_track['uri'])

    return list(artists_ids), list(song_titles), list(song_uri)

In [13]:
playlists = ['37i9dQZF1DWUZv12GM5cFk', '37i9dQZF1DXbITWG1ZJKYt', '37i9dQZF1DX9LbdoYID5v7', '2JajYJtwVXgEyzZqCyoL93', '37i9dQZF1DWTSKFpOdYF1r']

In [14]:
#creating empty lists to store data
all_song_titles = []
all_song_uris = []

#looping through playlists to get required information
for playlist in playlists:
    #extracting artists and artist ids
    artists = []
    artist_ids = []
    artists, artist_ids = get_artists_and_ids_from_playlist(playlist)

    #extracting songs and song ids
    songs = []
    song_ids = []
    songs, song_ids = get_songs_and_ids_from_playlist(playlist)

    #adding song data to the lists
    all_song_titles.extend(songs)
    all_song_uris.extend(song_ids)

    #extracting top songs from artists in the original playlist
    artists2 = []
    top_songs = []
    top_songs_uri = []
    artists2, top_songs, top_songs_uri = get_top_songs_from_artists_from_playlist(playlist)

    #adding top songs data to the lists
    all_song_titles.extend(top_songs)
    all_song_uris.extend(top_songs_uri)

#creating a dataframe from the accumulated data
spotify_df = pd.DataFrame({'song title': all_song_titles, 'song uri': all_song_uris})

#displaying the dataframe
spotify_df

,song title,song uri
0,Oops!...I Did It Again,spotify:track:6naxalmIoLFWR0siv8dnQQ
1,All The Small Things,spotify:track:2m1hi0nfMR9vdGC8UcrnwU
2,Breathe,spotify:track:3y4LxiYMgDl4RethdzpmNe
3,It's My Life,spotify:track:0v1XpBHnsbkCn7iJ9Ucr1l
4,Bye Bye Bye,spotify:track:62bOmKYxYg7dhrC6gH9vFn
...,...,...
7618,Requiem - 2005 Digital Remaster,spotify:track:0PZXhureTA1HfRkQWn2U1E
7619,The Wait - 2005 Digital Remaster,spotify:track:0t0GXXrMtTK1aXtVbnW00n
7620,Wardance - 2003,spotify:track:5YiaQ6kXLT2CJ46v9QKr19
7621,You'll Never Get To Me,spotify:track:7BrR1hSh7CxGuoCv4Ep1Xn
